# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

1. 取出今天所有的新聞
2. 取出現在時間兩小時內的新聞
3. 根據範例，取出三天前下午三點到五點的新聞

## 1. 取出今天所有的新聞

In [13]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.ettoday.net/news/news-list.htm'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')

for i in soup.find('div',class_='part_list_2').find_all('h3'):
    date = i.find('span',class_='date').text
    title = i.find('a').text
    print(date,title)

2020/01/02 09:43 快訊／黑鷹直升機疑墜落新北山區！民眾目擊嚇壞報警
2020/01/02 09:43 快訊／黑鷹直升機墜落　地點疑似宜蘭福山
2020/01/02 09:43 周曉涵新年剪「空氣短瀏海」激美❤　35歲萌樣根本逆生長！
2020/01/02 09:33 最棒新年禮！洋基王牌柯爾宣布當爸　等不及兒子6月報到
2020/01/02 09:28 差一字全不同！「抗菌、除菌、殺菌」功效差很大...1款防皮膚刺激
2020/01/02 09:24 《順風婦產科》67歲朴英奎結第4次婚了！　和女友熱戀4年「特選聖誕節結婚」
2020/01/02 09:13 女友與前夫勾勾纏　 醋男怒傳「你兒子該死啦」被判刑
2020/01/02 09:13 蔡英文再提4個認知　孫大千批：根本就是4個唬爛
2020/01/02 09:11 新台幣強升29字頭！2020開市首盤小貶0.4分　瞬間翻揚升破30元
2020/01/02 09:10 角落小夥伴洗衣袋呆萌又療癒！別買娃娃積灰塵　最實用款才必須收藏
2020/01/02 09:09 高雄跨年倒數出包！　韓國瑜還站台上…曾國城一秒垮臉超傻眼
2020/01/02 09:08 影片曝！亞裔大嬸「強扯83歲教宗手腕」　他反擊卻遭批道歉…輿論風向一面倒
2020/01/02 09:03 台股2020年開出紅盤！開盤上漲29點　站回1萬2千點
2020/01/02 09:01 韓國瑜「寶貝蛋」破了！深夜發文曝好消息　千人聽殼裡小雞啾啾叫：好療癒
2020/01/02 09:00 窗外就是東京鐵塔！特搜澀谷高CP值8大住宿　就連早餐也超好吃
2020/01/02 08:58 馬英九站台傅大偉　肯定20年基層實力是立委好人選
2020/01/02 08:53 搶銀樓金項鍊等20天再回去銷贓　笨賊：以為老闆會忘記我
2020/01/02 08:53 2020台指期開市上漲50點！　重新站回12000點之上
2020/01/02 08:39 藍營狂批《反滲透法》　賴清德：有滲透、違法行為才會觸法！
2020/01/02 08:37 張善政選舉公報沒博士學歷　蔡英文笑：我學位沒問題！論文是真的
2020/01/02 08:29 衝刺不分區　賴清德：國會不過半面對中國大陸挑戰很困難
2020/01/02 08:29 馬英九拿蘇宏達扯

## 2. 取出現在時間兩小時內的新聞

In [52]:
from datetime import datetime
now = datetime.now()
now_time = now.strftime('%Y-%m-%d %H:%M')

url = 'https://www.ettoday.net/news/news-list.htm'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')
for i in soup.find('div',class_='part_list_2').find_all('h3'):
    date = i.find('span',class_='date').text
    if datetime.strptime(date, '%Y/%m/%d %H:%M') - now_time
        
    title = i.find('a').text
    print(date,title)


SyntaxError: invalid syntax (<ipython-input-52-81c279ae4554>, line 10)

## 3. 根據範例，取出三天前下午三點到五點的新聞

'2020-01-02 10:04'